## Imports & settings

In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 120
plt.rcParams["savefig.dpi"] = 300

FIG_DIR = "report/figures"
os.makedirs(FIG_DIR, exist_ok=True)

TRAIN_PATH = "data/CW1_train.csv"
TEST_PATH  = "data/CW1_test.csv"
